In [71]:
import urllib.request
import time
from pyquery import PyQuery as pq
import csv
import re
from selenium import webdriver
import selenium.webdriver.chrome.service as srv
import time
import numpy as np
import pandas as pd
from textgenrnn import textgenrnn


In [72]:
def tr_lower(self):
    self = re.sub(r"İ", "i", self)
    self = re.sub(r"I", "ı", self)
    self = re.sub(r"Ç", "ç", self)
    self = re.sub(r"Ş", "ş", self)
    self = re.sub(r"Ü", "ü", self)
    self = re.sub(r"Ğ", "ğ", self)
    self = self.lower() # for the rest use default lower
    return self

In [73]:
def dowload(page,url,myset):
    offset = page * 50;
    urlx = f"{url}?pagingOffset={offset}&pagingSize=50"
    req = urllib.request.Request(
        url= urlx,
        data=None, 
        headers={
            'User-Agent': 'Mozilla/5.0 (Linux; Android 6.0.1; Nexus 5X Build/MMB29P) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2272.96 Mobile Safari/537.36 (compatible; Googlebot/2.1; +http://www.google.com/bot.html)'
        }
    )
    print(urlx)
    contents = urllib.request.urlopen(req).read()
    if get(contents,myset):
        return
    page +=1
    dowload(page,url,myset)

In [74]:
def get(cont,myset):
    d = pq(cont)
    lic = d('.searchResultsRowClass')
    e = lic('a')
    numbefore = len(myset)
    for x in e:
        myset.add(x.attrib['href'])
    numafter = len(myset)
    print(len(myset))
    return numbefore == numafter

In [83]:
def getData(name,myset):
    service = srv.Service('./chromedriver.exe')
    service.start()
    capabilities = {'chrome.binary': '/path/to/custom/chrome'}
    driver = webdriver.Remote(service.service_url, capabilities)
    result = []
    counter = 0
    for url in myset:
        counter +=1
        if counter >5000:
            break
        try:
            driver.get('https://www.sahibinden.com'+url);
            desc=driver.find_element_by_id('classifiedDescription').get_attribute('innerText')
            desc = desc.replace('-',' ')
            desc = desc.replace('*',' ')
            desc = desc.replace('"','')
            desc = desc.replace("'",' ')
            desc = desc.replace('+',' ')
            desc = desc.replace('/',' ')
            desc = desc.replace('*',' ')
            desc = desc.replace('..',' ')
            desc = desc.replace('...',' ')
            desc = desc.replace('_',' ')
            desc = re.sub('\s+', ' ', desc).strip()
            desc =  tr_lower(desc)

            title=driver.find_element_by_css_selector('.classifiedDetailTitle h1').get_attribute('innerText')
            title = title.replace('\n','')
            title = title.replace('\t','')
            title = title.replace('-',' ')
            title = title.replace('+',' ')
            title = title.replace('/',' ')
            title = title.replace('"','')
            title = title.replace('*',' ')
            title = title.replace('..',' ')
            title = title.replace('...',' ')
            title = title.strip('"')
            title = re.sub('\s+', ' ', title).strip()
            title= tr_lower(title)
            result.append({ "desc": desc, "title": title})
            print(title)
        except:
            print('hata olustu' + url)
        time.sleep(1)
    driver.quit()
    with open(name+'Titles.csv', 'a',newline="\n", encoding='utf-8') as csvfile:
        fieldnames = ['title']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for l in result:
            writer.writerow({'title': l["title"]})
    with open(name+'Desc.csv', 'a',newline="\n", encoding='utf-8') as csvfile:
        fieldnames = ['desc']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        for l in result:
            writer.writerow({'desc': l["desc"]})

In [ ]:

service = srv.Service('./chromedriver.exe')
service.start()
capabilities = {'chrome.binary': '/path/to/custom/chrome'}
driver = webdriver.Remote(service.service_url, capabilities)
try:
    driver.get('https://www.sahibinden.com/kategori/otomobil')
    titles = driver.find_elements_by_css_selector('.categoryList .jspPane li')
    brands_links = []
    for x in titles:
        brand = x.find_element_by_css_selector('a').get_attribute('innerText')
        count = (int)(x.find_element_by_css_selector('span').get_attribute('innerText').replace('(','').replace(')','').replace('.',''))
        print(f'found {count} on {brand}')
        if count >100:
            link = x.find_element_by_css_selector('a').get_attribute('href')
            brands_links.append({'link':link,'brand':brand})
        if brand == 'Volvo':
            break
    for brnd in brands_links:
        myset = set()
        driver.get(brnd['link'])
        titles = driver.find_elements_by_css_selector('#searchCategoryContainer .jspPane li a')
        for x in titles:
            link = x.get_attribute('href')
            print("getting " + link)
            dowload(0,link,myset)
        getData(brnd['brand'],myset)
except:
    raise

found 1 on Acura
found 841 on Alfa Romeo
found 11 on Anadol
found 59 on Aston Martin
found 10243 on Audi
found 85 on Bentley
found 18551 on BMW
found 1 on Brilliance
found 1 on Bugatti
found 10 on Buick
found 51 on Cadillac
found 74 on Chery
found 3263 on Chevrolet
found 323 on Chrysler
found 6884 on Citroën
found 3763 on Dacia
found 121 on Daewoo
found 130 on Daihatsu
found 29 on Dodge
found 206 on DS Automobiles
found 121 on Ferrari
found 22553 on Fiat
found 1 on Fisker
found 23240 on Ford
found 126 on Geely
found 9615 on Honda
found 16456 on Hyundai
found 9 on Ikco
found 41 on Infiniti
found 391 on Jaguar
found 2823 on Kia
found 1 on Koenigsegg
found 1 on Kral
found 900 on Lada
found 46 on Lamborghini
found 99 on Lancia
found 7 on Lexus
found 20 on Lincoln
found 6 on Lotus
found 1 on Marcos
found 150 on Maserati
found 1328 on Mazda
found 2 on McLaren
found 16921 on Mercedes - Benz
found 2 on Mercury
found 9 on MG
found 859 on Mini
found 656 on Mitsubishi
found 2 on Moskwitsch
found 

2014 giulietta (hata, boya, değişen, tramer vs. yok)
temiz romeo
alfa romeo 156 2.0 i
2008 model temiz alfa romeo
acil memurdan temiz full 156 ihtiyaçtan satılık
acilllll satılık
acil!!! alfa romeo gıulıetta super tct 1.6 jtd otomatik
alfa romeo 156 1999
ılk sahibinden degisensiz hasarsiz boyasiz giulietta 1.6 dizel
temiz giulietta 1.6 jtd dizel cam tavan sanruf full full
sahibinden fiyat düştü full ful170hp kazasız değişensiz boyasız
pnr auto 2008 alfa romeo gt 1.9 dizel boyasız emsalsiz 20.150km
mühendisten 120 hp 147
166 v6 turbo acil
alfa romeo 156 binilip görülecek araç sahibinden
italyan aygırı
deri koltuk, sunroof
sahibinden fırsat aracı
ayaklarınız yerden kesilecek değışensiz hasar kayıtzz.
doktordan ilk sahibinden temiz kullanılmış gulietta
kazasız tertemiz alfa romeo mito 1.6 dizel 120hp
bursaya hayırlı olsun
ilk sahibinden 1.4 170hp, tct, cam tavan, spor paket, f1
çok temiz bakımları yeni yapılmış 159 1.9 jtd dıstınctıve
24000 km.de temiz, boseli spor koltuklu orjinal jantlı

alfa gıulıetta 2016 süper tct 1.6 jtd dizel otomatik !
vefa citroen ana bayi 2002 alfa romeo 156 1.6
2017 alfa romeo giulietta 1.6jtd progression sunroof boyasız
2004 model lpg li sunrooflu temiz 156
eğitim için (üzülerek) satılık 159 distinctive plus
2021 yılına kadar muayeneli temiz alfa 146 ts
alfa romeo 159 1,9 jts
kara yılan ayrılık vakti .117 fav teşekkürler son 3 gün !
full sıfır ayarında acill fiyat düştü
sahibinden temiz giulietta 2014
sahibinden temiz alfa
oto stop'tan 'türkiye'de tek' alfa romeo 159 sw jtdm
türkiye'de tek acil satılık
2004 model alfa romeo 156 1.6 ts distinctive hatasız
hata boya yok
2013 model ilk gelen alır. fiyat düştü.
2014 model 67.000kmde alfaromeo mito
sahibinden temiz 29 binden düştü
italyan yakışıklısı
sahibinden satılık alfa romeo
alfa romeo 166 ts 155 hp e segment servis bakımlı değişen yok
alfa romeo
böylesi yok!dizel otomatik değisensiz masrafsız extralı
temiz alfa 156 1.6 2000 model
bilgiçten full emsalsiz masrafsiz
2006 model mk2 temiz sorunsu

alfa romeo 156 2.0ts yakışıklı twinspark
hatasız kazasız giulietta 1.6 distinctive
2008 benzin lpg li 195 bin km de siyah
alfa center'dan 2009 159 1.9 jtd dist.plus q tronic,deri,full
sahibinden alfa romeo 156
sahibinden alfa
2012 giulietta cam tavan distinctive spor 1 paket
sahibinden guiletta
alfa romeo takas olur panelvan araçlarla
bakımlı yılan
sahibinden temiz alfa 147
müthiş temizlikte. . . lütfen açıklamaları okuyunuz
alfa modena'dan 2003 147 selespeed sunroof deri döşeme bose
hatasız mermi distivince
garantîlî araç full servis bakımlı
acill satılık tertemiz alfa romeo 1.6 jtd
2002 alfa romeo 156 2.0 ts
tramersiz, değişensiz, cam tavanlı, sigara içilmemiş giulietta
alfa romeo giulietta 1.6 super tct
biniciye gidecek alfa romeo
sahibinden 156
ihtiyaçtan satılık
alfa modena'dan 2001 selespeed sportwagon emsalsiz temizlikte
2002 model alfa romeo 156 1.6 ts lpg li
go car 2001 alfa romeo 156 twin spark 217.000 km
2012 1.6jtd dizel distinctive spor paket cam tavan, 146.000 km
karbon p

üstün performans yakit cimrisi temiz
yakışıklı romeo ilanda !!!
yeni yılın muhteşem arabası alfa romeo julıetta
çetinkaya'dan 2014 giulieatta 1.6 dizel boyasız tramersiz 93 km
diş hekiminden orjinal koleksiyonluk
askere gideceğim için satıyorum
arel 2012 alfa giulietta 1.4tb multiair cam tavan otomatik
2016 otomatik giulietta 44.000 km
ismet otomotivden 2004 alfa romeo 156 1.6 ts progression lpg'li
1996 alfa 145 1.6 boxer motor göz alıcı güzellikte kaçmaz !!!
son fiyatıdır. en dolusudur. 1.6 multijet dizel motor.
beyaz inci 2012
pazarlık var takas olur
sahibinden satılık
' bks motors ' 2012 giulietta otomatik 170 bg cam tavan
doktor bayan'dan giulietta
kazasız değişensiz giulietta
sıfır multiair tct
italyan aygırı alfa romeo
orjinal 122.000 km.'de çok temiz italyan aygırı
ihtiyaçtan dolayı satılık temiz kullanılmış
km. yanıltmasın sıfır motor full bakımlı
giulietta 2017 model 22 bin super tct dizel otm.
yetkili bayi'den alfa romeo mito 1.3 multıjet dizel manuel
acil satılık bakımlı
alf

...

eşedosta gider, sorunsuz 2011 model 2013 trafiğe çıkış, kusursuz
yeniköy motors izmir 2013 audi a8 3.0 tdı quattro long bayi
2017 audi a6 2.0tfsi 4x4
böylesi birdaha gelmez
oto derin den 2012 audı a8 long 3.0 tdı quattro bayı çıkışlı
doğruyol auto'dan hatasız emsalsiz 4k ıst sunroof doğuş içi bej
garantili sıfır motor audı a6 2.0tdı multıtronıc
özde otomotiv den tertemiz hasarsız boyasız audi a6
acil ! temiz a3 lansman renk camtavan sunroof full s line
2018 a5 sportback 1.4 tfsı sport delüx prestige ıstm hfza a.klma
serkan otomotiv 'den audi a6 2.0 tdı 177 hp doğuş cıkışlı 2011
audi a3 garaj arabası 2016 modeldir !!!
otomoto garaj 2015 audi.a5 coupe 2.0 tdi.quattro 190bg boyasız
2013 audi a5 2.0 tdi otomatik 177 hp bayi çıkışlı
audi a1 tramersiz sıfır ayarında sahibinden
2018 audı a5 sportback 1.4 tfsı 6.850km füme bej deri hatasız
alamancidan audi80b2 benzin& lpg klima var
temiz sahibinden multitronic a4 2.0tdi
erman oto'dan 2012 model iç dış slıne 177hp hatasız boyasız audi
acil sahi

hatasz emsalsiztemz 2010 içi kahvederi sunroof çifthafıza 2.0tdi
2010 audi a5 quattro 75.700 km s line f1 camtavan hafıza ısıtma
ekcar'dan 2018 audı a3 sedan 1.6 tdı 116 hp design s tronıc
ilk sahibinden,en dolusu,düşük km a4
attarlar otomotiv'den hatasız garantili a5 prestige delux paket
temiz sorunsuz a4 sedan 70.000 km
sahibinden orjinal a3
yeni kasa temiz
2014 model a5 quattro coupe boyasız 190'lık tertemiz
audi a4 satışta son fiyat
sahibinden temiz araç
sunrooflu ful ful audı a1
ekşiler otomotivden 2010 audi a3 sportback 1.4 tfsı otomatik
araç fazlalığından kupon a5(meraklısına)
vade takas audi a 3 1.6 tdi otomatik
cam tavan bay & bayan beraber kullanılabilecek yer uçağı
fuul x fuul a 4 otomotık dizel garantılı
tr,tek 1.6td turbo dizel hatasız 4 lastik vize yeni
red car istanbul 2009 cabrio 1.8t 160 hp
orjinal audi a4 sedan 1.8 tfsı multitronic(otomatik) 160 hp
son fiyat acil l l 2018 model 8600 km hatasız sahibinden
mas auto,2011,audi a1,s line full,mavi,otomatik,kazasız,80bin km

audi a5 108.500km otomatik (8 ileri) boyasız kazasiz çok temiz
yalçın'dan 2011 hatasız audi a4 2.0 tdi multitronic
hata olustu/ilan/vasita-otomobil-audi-boyasiz-160-lik-otomatik-full-takasli-630549824/detay
audi a4 quattro 2.0 motor temiz araç
2018 audı a6 2.0 tfsı quattro s tronic
çenet otomotivden audi
autolıfe 2016 a6 2.0 tdı vakum spor direksiyon navıgasyon %18kdv
autopark otomotivden audı a8 long 3.2 fsı
2015 model a3 attraction hatasız 32.000km !!!
cam gibi gel bak al git
orjinal full düşük km audi a3
sahibinden satılıktır
asfalt canavarı audı a6 3.0 tdı quatro 4 4 yer uçağı 273 beygir
2017 audi has otomotiv den a5 2.0 tdi quattro 190 beygir
makam aracı,245 beygir,audı a6 3.0 tdi quattro 4x4 pazarlıksızzz
2016 model audi a3 1.6 tdı ambiente s tronic cam tavan led xenon
sahibinden 2018 model 0 km audi a3 sedan
sadece 31.500 km. sahibinden a5 2.0 tdi sportback 2014
orjinal masrafsız ekonomik düz vites
audı 1.8 değişensiz kazasız masrafsız
2011 model sahibinden satılık kırmızı a1
ka

günay otomotiv den 2014 audi a8 4.2l tdı quattro bayii çıkışlı
2014 yeni kasa audı a6 avant 3.0 tdı quattro tıptronıc 272 hp
kayzen'den 2014 a5 2.0 tdi sportback 57 bin de deri sunroof full
ac motors a3 sedan cam tavan yeni kasa desing line boyasız
2012 a5 2.0tdı 177hp doğuş çıkış sunroof deri ısıtma hasar kayıt
audı a7 sportback 50 tdı quattro tıptronıc
temiz kullanılmış 2014 trafiğe çıkışlı , ihtiyaçtan satılık
kazasız, boyasız, değişensiz, tertemiz 2013 trafiğe çıkışlı
öğretmenden,servis bakımlı,acil
audi rs4 70 bin km de xneon sanruf deri isitma
audi a5 2.0 tdi navigasiyon dvd tv multimedya 15 gb harddisk usb
asım oto audı a5 2.0 tdı quattro bayi çıkışlı 36,800 km
özdağ auto
peugeot deniz 2.el yetkili bayiden audı a4 2.0 tdı
karataş'tan 2013 audı a5 sportback 2.0 tdi.quattro s line doğuş
2013 çıkışlı ilk sahibinden çift hafıza üç kol full full
2006 model otomatik lpgli 5 kapılı orjinal audi a3
özkanoğulları .2013 audı a4 2.0 tdı quattro 177 hp hatasız
fiyat düştüüüü 2.sahibinden te

audi a3 muayeli 2021 yürüyeni iyi alıcısına hayırlı olsun
sahibinden audi a5 quattro 190 hp s tronic
sahibinden satılık araba
185.000 kmde hatasız boyasız acil
2. sahibinden orj km çok temiz 2007
2018 aralık çıkışlı hatasız cosmos mavisi 1.6 tdı manuel
2015 model 86.000km de audi a3 1.6 tdi cam tavan xenon
doğuş oto audı 2.0 tdı 143 bg beyaz içi bej sunroff deri ısıtma
audi a4 1.8 120 hp s line f1 led xenon sunroof 3 kol direksiyon
yeni kasa 177ps sunroof led deri katlanır ayna
acill 2015 audı a5 quattro sportback
sahibinden sıfır motor değişensiz audi a6
sahibinden satılık a4
boyasız değişensiz otomatik cam tavan
düşük km a6
boyasız makyajlı audi a3 f1 vites!!
1.9 tdı full full sıfır masraf
piranlar audi a7 sportback biturbo 3.0 tdı quattro hatasız
2018 audı a5 sback 1.4 tfsı sport çift paket stronic hatasız
sahibinden bayii 313bg audi a7 hatasız
audi a3 1.6 tdı otomatik
ugur auto'dan 2011 bayi boyasız vakum ön arka kamera 4k ısıtma
topçu auto 2013 model audi a3 1.6tdi ambition cam ta

In [80]:
def generateText(nm)
    textgen = textgenrnn(name=nm)
    textgen.reset()
    textgen.train_from_file(nm+'Desc.csv',
                            new_model=True,
                            rnn_bidirectional=True,
                            word_level=True,
                            rnn_size=64,
                            dim_embeddings=300,
                            num_epochs=3)

    print(textgen.model.summary())

5,784 texts collected.
Training new model w/ 2-layer, 64-cell Bidirectional LSTMs
Training on 554,950 word sequences.
Epoch 1/3


 312/4335 [=>............................] - ETA: 5:27:49 - loss: 9.21 - ETA: 3:05:56 - loss: 9.15 - ETA: 2:18:13 - loss: 10.084 - ETA: 1:54:12 - loss: 9.966 - ETA: 1:39:58 - loss: 9.69 - ETA: 1:30:19 - loss: 9.44 - ETA: 1:23:17 - loss: 9.27 - ETA: 1:18:07 - loss: 9.05 - ETA: 1:14:12 - loss: 8.90 - ETA: 1:11:10 - loss: 8.79 - ETA: 1:08:29 - loss: 8.66 - ETA: 1:06:20 - loss: 8.61 - ETA: 1:04:26 - loss: 8.53 - ETA: 1:02:50 - loss: 8.45 - ETA: 1:01:28 - loss: 8.39 - ETA: 1:00:16 - loss: 8.34 - ETA: 59:16 - loss: 8.2946 - ETA: 58:20 - loss: 8.25 - ETA: 57:27 - loss: 8.18 - ETA: 56:39 - loss: 8.14 - ETA: 55:58 - loss: 8.09 - ETA: 55:20 - loss: 8.04 - ETA: 54:45 - loss: 8.01 - ETA: 54:13 - loss: 7.97 - ETA: 53:42 - loss: 7.94 - ETA: 53:11 - loss: 7.91 - ETA: 52:45 - loss: 7.88 - ETA: 52:23 - loss: 7.86 - ETA: 52:01 - loss: 7.83 - ETA: 51:39 - loss: 7.82 - ETA: 51:22 - loss: 7.80 - ETA: 51:03 - loss: 7.78 - ETA: 50:46 - loss: 7.76 - ETA: 50:28 - loss: 7.74 - ETA: 50:14 - loss: 7.70 - ETA: 50:

1254/4335 [=======>......................] - ETA: 42:00 - loss: 5.18 - ETA: 41:59 - loss: 5.18 - ETA: 41:59 - loss: 5.18 - ETA: 41:58 - loss: 5.17 - ETA: 41:57 - loss: 5.17 - ETA: 41:56 - loss: 5.17 - ETA: 41:56 - loss: 5.17 - ETA: 41:55 - loss: 5.17 - ETA: 41:54 - loss: 5.17 - ETA: 41:54 - loss: 5.17 - ETA: 41:53 - loss: 5.17 - ETA: 41:52 - loss: 5.17 - ETA: 41:51 - loss: 5.17 - ETA: 41:51 - loss: 5.17 - ETA: 41:50 - loss: 5.16 - ETA: 41:49 - loss: 5.16 - ETA: 41:48 - loss: 5.16 - ETA: 41:48 - loss: 5.16 - ETA: 41:47 - loss: 5.16 - ETA: 41:46 - loss: 5.16 - ETA: 41:45 - loss: 5.16 - ETA: 41:45 - loss: 5.16 - ETA: 41:44 - loss: 5.16 - ETA: 41:43 - loss: 5.16 - ETA: 41:43 - loss: 5.16 - ETA: 41:42 - loss: 5.16 - ETA: 41:41 - loss: 5.16 - ETA: 41:40 - loss: 5.15 - ETA: 41:40 - loss: 5.15 - ETA: 41:39 - loss: 5.15 - ETA: 41:38 - loss: 5.15 - ETA: 41:37 - loss: 5.15 - ETA: 41:37 - loss: 5.15 - ETA: 41:36 - loss: 5.15 - ETA: 41:35 - loss: 5.15 - ETA: 41:35 - loss: 5.15 - ETA: 41:34 - loss: 

1568/4335 [=========>....................] - ETA: 38:04 - loss: 4.98 - ETA: 38:04 - loss: 4.98 - ETA: 38:03 - loss: 4.98 - ETA: 38:02 - loss: 4.98 - ETA: 38:02 - loss: 4.98 - ETA: 38:01 - loss: 4.98 - ETA: 38:00 - loss: 4.98 - ETA: 38:00 - loss: 4.98 - ETA: 37:59 - loss: 4.98 - ETA: 37:58 - loss: 4.98 - ETA: 37:57 - loss: 4.98 - ETA: 37:57 - loss: 4.97 - ETA: 37:56 - loss: 4.97 - ETA: 37:55 - loss: 4.97 - ETA: 37:54 - loss: 4.97 - ETA: 37:54 - loss: 4.97 - ETA: 37:53 - loss: 4.97 - ETA: 37:52 - loss: 4.97 - ETA: 37:51 - loss: 4.97 - ETA: 37:51 - loss: 4.97 - ETA: 37:50 - loss: 4.97 - ETA: 37:49 - loss: 4.97 - ETA: 37:48 - loss: 4.97 - ETA: 37:47 - loss: 4.97 - ETA: 37:47 - loss: 4.97 - ETA: 37:46 - loss: 4.97 - ETA: 37:45 - loss: 4.97 - ETA: 37:44 - loss: 4.97 - ETA: 37:43 - loss: 4.97 - ETA: 37:43 - loss: 4.97 - ETA: 37:42 - loss: 4.97 - ETA: 37:41 - loss: 4.97 - ETA: 37:40 - loss: 4.97 - ETA: 37:39 - loss: 4.97 - ETA: 37:39 - loss: 4.97 - ETA: 37:38 - loss: 4.97 - ETA: 37:37 - loss: 

1882/4335 [============>.................] - ETA: 35:08 - loss: 4.84 - ETA: 35:07 - loss: 4.84 - ETA: 35:07 - loss: 4.84 - ETA: 35:06 - loss: 4.84 - ETA: 35:06 - loss: 4.84 - ETA: 35:06 - loss: 4.84 - ETA: 35:06 - loss: 4.84 - ETA: 35:05 - loss: 4.84 - ETA: 35:05 - loss: 4.84 - ETA: 35:04 - loss: 4.84 - ETA: 35:04 - loss: 4.84 - ETA: 35:04 - loss: 4.84 - ETA: 35:04 - loss: 4.84 - ETA: 35:03 - loss: 4.84 - ETA: 35:03 - loss: 4.84 - ETA: 35:03 - loss: 4.84 - ETA: 35:02 - loss: 4.84 - ETA: 35:02 - loss: 4.84 - ETA: 35:02 - loss: 4.84 - ETA: 35:01 - loss: 4.84 - ETA: 35:01 - loss: 4.84 - ETA: 35:01 - loss: 4.84 - ETA: 35:00 - loss: 4.84 - ETA: 35:00 - loss: 4.84 - ETA: 34:59 - loss: 4.84 - ETA: 34:59 - loss: 4.84 - ETA: 34:59 - loss: 4.84 - ETA: 34:58 - loss: 4.84 - ETA: 34:58 - loss: 4.84 - ETA: 34:57 - loss: 4.83 - ETA: 34:57 - loss: 4.83 - ETA: 34:57 - loss: 4.83 - ETA: 34:56 - loss: 4.83 - ETA: 34:56 - loss: 4.83 - ETA: 34:56 - loss: 4.83 - ETA: 34:55 - loss: 4.83 - ETA: 34:55 - loss: 

...

3454/4335 [======================>.......] - ETA: 13:14 - loss: 3.08 - ETA: 13:13 - loss: 3.08 - ETA: 13:13 - loss: 3.08 - ETA: 13:12 - loss: 3.08 - ETA: 13:11 - loss: 3.08 - ETA: 13:11 - loss: 3.08 - ETA: 13:10 - loss: 3.08 - ETA: 13:09 - loss: 3.08 - ETA: 13:09 - loss: 3.08 - ETA: 13:08 - loss: 3.08 - ETA: 13:07 - loss: 3.08 - ETA: 13:07 - loss: 3.08 - ETA: 13:06 - loss: 3.08 - ETA: 13:05 - loss: 3.08 - ETA: 13:05 - loss: 3.08 - ETA: 13:04 - loss: 3.08 - ETA: 13:03 - loss: 3.08 - ETA: 13:03 - loss: 3.08 - ETA: 13:02 - loss: 3.08 - ETA: 13:01 - loss: 3.08 - ETA: 13:01 - loss: 3.08 - ETA: 13:00 - loss: 3.08 - ETA: 12:59 - loss: 3.08 - ETA: 12:59 - loss: 3.08 - ETA: 12:58 - loss: 3.08 - ETA: 12:57 - loss: 3.08 - ETA: 12:57 - loss: 3.08 - ETA: 12:56 - loss: 3.08 - ETA: 12:55 - loss: 3.08 - ETA: 12:55 - loss: 3.08 - ETA: 12:54 - loss: 3.08 - ETA: 12:53 - loss: 3.08 - ETA: 12:53 - loss: 3.08 - ETA: 12:52 - loss: 3.08 - ETA: 12:51 - loss: 3.08 - ETA: 12:51 - loss: 3.08 - ETA: 12:50 - loss: 

3768/4335 [=========================>....] - ETA: 9:45 - loss: 3.090 - ETA: 9:45 - loss: 3.090 - ETA: 9:44 - loss: 3.090 - ETA: 9:43 - loss: 3.090 - ETA: 9:43 - loss: 3.090 - ETA: 9:42 - loss: 3.090 - ETA: 9:41 - loss: 3.090 - ETA: 9:41 - loss: 3.090 - ETA: 9:40 - loss: 3.090 - ETA: 9:39 - loss: 3.090 - ETA: 9:39 - loss: 3.090 - ETA: 9:38 - loss: 3.090 - ETA: 9:37 - loss: 3.090 - ETA: 9:37 - loss: 3.090 - ETA: 9:36 - loss: 3.090 - ETA: 9:35 - loss: 3.090 - ETA: 9:35 - loss: 3.090 - ETA: 9:34 - loss: 3.090 - ETA: 9:33 - loss: 3.090 - ETA: 9:33 - loss: 3.090 - ETA: 9:32 - loss: 3.090 - ETA: 9:31 - loss: 3.090 - ETA: 9:31 - loss: 3.090 - ETA: 9:30 - loss: 3.090 - ETA: 9:29 - loss: 3.090 - ETA: 9:29 - loss: 3.090 - ETA: 9:28 - loss: 3.090 - ETA: 9:27 - loss: 3.090 - ETA: 9:27 - loss: 3.090 - ETA: 9:26 - loss: 3.090 - ETA: 9:25 - loss: 3.090 - ETA: 9:25 - loss: 3.090 - ETA: 9:24 - loss: 3.090 - ETA: 9:23 - loss: 3.090 - ETA: 9:23 - loss: 3.090 - ETA: 9:22 - loss: 3.090 - ETA: 9:21 - loss: 3

4082/4335 [===========================>..] - ETA: 6:17 - loss: 3.094 - ETA: 6:16 - loss: 3.094 - ETA: 6:15 - loss: 3.094 - ETA: 6:15 - loss: 3.094 - ETA: 6:14 - loss: 3.094 - ETA: 6:13 - loss: 3.094 - ETA: 6:13 - loss: 3.094 - ETA: 6:12 - loss: 3.094 - ETA: 6:11 - loss: 3.094 - ETA: 6:11 - loss: 3.094 - ETA: 6:10 - loss: 3.094 - ETA: 6:09 - loss: 3.094 - ETA: 6:09 - loss: 3.094 - ETA: 6:08 - loss: 3.094 - ETA: 6:07 - loss: 3.094 - ETA: 6:07 - loss: 3.094 - ETA: 6:06 - loss: 3.094 - ETA: 6:05 - loss: 3.094 - ETA: 6:05 - loss: 3.094 - ETA: 6:04 - loss: 3.094 - ETA: 6:03 - loss: 3.094 - ETA: 6:03 - loss: 3.094 - ETA: 6:02 - loss: 3.094 - ETA: 6:01 - loss: 3.094 - ETA: 6:01 - loss: 3.094 - ETA: 6:00 - loss: 3.094 - ETA: 6:00 - loss: 3.094 - ETA: 5:59 - loss: 3.094 - ETA: 5:58 - loss: 3.094 - ETA: 5:58 - loss: 3.094 - ETA: 5:57 - loss: 3.094 - ETA: 5:56 - loss: 3.094 - ETA: 5:56 - loss: 3.094 - ETA: 5:55 - loss: 3.095 - ETA: 5:54 - loss: 3.095 - ETA: 5:54 - loss: 3.095 - ETA: 5:53 - loss: 3

4335/4335 [==============================] - ETA: 2:48 - loss: 3.079 - ETA: 2:47 - loss: 3.079 - ETA: 2:46 - loss: 3.079 - ETA: 2:46 - loss: 3.079 - ETA: 2:45 - loss: 3.078 - ETA: 2:44 - loss: 3.078 - ETA: 2:44 - loss: 3.078 - ETA: 2:43 - loss: 3.078 - ETA: 2:42 - loss: 3.078 - ETA: 2:42 - loss: 3.078 - ETA: 2:41 - loss: 3.078 - ETA: 2:40 - loss: 3.078 - ETA: 2:40 - loss: 3.078 - ETA: 2:39 - loss: 3.078 - ETA: 2:38 - loss: 3.077 - ETA: 2:38 - loss: 3.077 - ETA: 2:37 - loss: 3.077 - ETA: 2:36 - loss: 3.077 - ETA: 2:36 - loss: 3.077 - ETA: 2:35 - loss: 3.077 - ETA: 2:34 - loss: 3.077 - ETA: 2:34 - loss: 3.077 - ETA: 2:33 - loss: 3.077 - ETA: 2:32 - loss: 3.077 - ETA: 2:32 - loss: 3.077 - ETA: 2:31 - loss: 3.077 - ETA: 2:30 - loss: 3.076 - ETA: 2:30 - loss: 3.076 - ETA: 2:29 - loss: 3.076 - ETA: 2:28 - loss: 3.076 - ETA: 2:28 - loss: 3.076 - ETA: 2:27 - loss: 3.076 - ETA: 2:26 - loss: 3.076 - ETA: 2:26 - loss: 3.076 - ETA: 2:25 - loss: 3.076 - ETA: 2:24 - loss: 3.076 - ETA: 2:24 - loss: 3

####################
" aracımız comfort paket olup içi dışı tertemiz olup deforme yoktur . yedek anahtarı ve kitapçıkları vardır . tüm bakımları eksiksiz yapılmıştır .

aracım borusan çıkışlı olup servis bakımlıdır . 2 parça boya vardır . hasar kaydı yoktur . "

" araç model bilgisi ; bmw 5 . 20i executıve m sport plus 1 . 6 motor 170 hp kasko bedeli : 130 . 000 tl tramer kaydı vardır . vakumlu kapılar hayalet ekran vakumlu kapılar elektrikli bagaj elektrikli hafızalı koltuk koltuk ısıtma geri görüş kamerası ön arka park sensörü ön arka park sensörü elektrikli arka perde elektrikli katlanır dış dikiz aynaları elektrikli perde ön ve arka park sensörü elektrikli katlanır yan aynalar elektrokrom iç ayna ön arka kol dayama park mesafe kontrol ön arka sis farı far sensörü yağmur sensörü otomatik kararan dikiz aynası otomatik kapı kilit fonksiyonu yol bilgisayarı 4 farklı sürüş modu seçeneği auto hold sistemi ön ve arka kol dayama ön ve arka park mesafe kontrol elektrikli arka perde elektrok

In [83]:
textgen.generate(300, temperature=0.4)


  0%|                                                                                          | 0/300 [00:00<?, ?it/s]

" 2014 model bmw 5 . 25d xdrive premium borusan çıkışlı dış renk : beyaz iç renk : bej deri koltuk ısıtma sunroof hayalet gösterge vakumlu kapılar elektrikli bagaj hayalet ekran vakumlu kapılar elektrikli bagaj kapağı geri görüş kamerası elektrikli arka perde elektrikli hafızalı ısıtmalı ön koltuklar elektrikli direksiyon ön arka park sensörü elektrikli arka perde çift yönlü dijital klima elektrikli arka perde yan makam perdeleri elektrikli arka perde elektrikli arka perde yağmur sensörü far sensörü far yıkama ön arka park sensörü far sensörü yağmur sensörü usb aux giriş çıkış krom egzoz başlıkları bmw logolu kapı kolları led aydınlatma sis farı arka park sensörü yağmur sensörü far gece sensörü park sensörü yağmur sensörü far sensörü otomatik kararan dikiz aynası yokuş kalkış desteği çift yönlü dijital klima yol bilgisayarı yağmur sensörü far sensörü ön sis farları far yıkama ön arka kol dayama park sensörü yağmur sensörü far sensörü ön arka park sensörü fonksiyonel direksiyon simidi ö

  0%|▎                                                                                 | 1/300 [00:01<07:44,  1.55s/it]

" araç bilgileri 2011 model bmw 5 . 20d premium m sport paket sunroof geri görüş kamerası elektrikli hafızalı ısıtmalı koltuklar elektrikli ön koltuklar ısıtmalı ön koltuklar elektrikli direksiyon ön ve arka park sensörü geri görüş kamerası bluetooth telefon usb aux elektrikli arka perde yan protokol perde yan makam perdeleri elektrikli arka perde elektrikli katlanır ayna yağmur sensörü far sensörü ön arka park sensörü far yıkama ön arka kol dayama isofix aracımız yetkili servis bakımlıdır . aracımızda 1 parça boya mevcuttur . tramer kaydı yoktur .



  1%|▌                                                                                 | 2/300 [00:02<06:39,  1.34s/it]

...

 99%|███████████████████████████████████████████████████████████████████████████████▏| 297/300 [06:24<00:01,  1.89it/s]

araç borusan çıkışlı olup yeni yapılmıştır .

" sahibinden satılık bmw 5 . 20i premium bayi çıkışlı borusan bakımlı hatasız boyasız hasar kayıtsız borusan çıkışlı sunroof bej deri döşeme elektrikli hafızalı koltuk elektrikli arka perde elektrikli arka perde geri görüş kamerası bluetooth telefon bağlantısı geri görüş kamerası bluetooth telefon bağlantısı auto hold elektrikli park freni yokuş kalkış desteği elektrikli katlanır yan aynalar elektrokrom iç ayna yağmur sensörü far sensörü ön ve arka park mesafe sensörü yağmur sensörü far sensörü otomatik kararan dikiz aynası far yıkama çift bölgeli dijital klima start stop bluetooth telefon bağlantısı usb aux girişi bluetooth telefon bağlantısı fonksiyonel deri direksiyon arka kol dayama ısofıx 17 alaşım jantlar sis farı



100%|███████████████████████████████████████████████████████████████████████████████▋| 299/300 [06:25<00:00,  1.84it/s]

" borusan çıkışlı , borusan bakımlı , hayalet gösterge , vakumlu kapılar , elektrikli bagaj , hayalet gösterge , vakumlu kapılar , elektrikli bagaj , hafızalı koltuk , elektrikli bagaj , hayalet ekran , vakumlu kapılar , elektrikli hafızalı direksiyon , elektrikli direksiyon , ısıtmalı hafızalı koltuk , koltuk ısıtma , elektrikli hafızalı koltuklar , elektrikli direksiyon , hafızalı sürücü koltuğu , elektrikli ön koltuklar , ısıtmalı ön koltuklar , hafızalı sürücü koltuğu , elektrikli , ısıtmalı , hafızalı , katlanır , ısıtmalı ön koltuklar , elektrikli katlanır ayna , elektrokrom iç ve dış aynalar , far yıkama , bluetooth , aux , usb , girişi , cd çalar , anahtarsız çalıştırma , start & stop , stop , anahtarsız çalıştırma sis farı , yağmur sensörü , far sensörü , kararan iç dikiz aynası abs , asr , esp , radyo cd çalar , yol bilgisayarı , fonksiyonel deri direksiyon , alarm , ımmobılızer , elektrikli , ısıtmalı aynalar , elektrikli ön ve arka camlar , uzaktan kumandalı merkezi kilit ,

100%|████████████████████████████████████████████████████████████████████████████████| 300/300 [06:27<00:00,  1.06it/s]


In [18]:
textgen.generate(3, temperature=1.0)

  0%|                                                                                            | 0/3 [00:00<?, ?it/s]

bmw 3 20 20d li̇

2014 bmw " hatasiz boyasiz en deri̇ hi̇z gülan



 67%|████████████████████████████████████████████████████████                            | 2/3 [00:00<00:00, 10.86it/s]

emsalsiz sanroof askeri altı hatasiz aci̇lll si̇tede boya led 143 hayalet paket



100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00,  9.13it/s]


In [ ]:
textgen.generate(3, temperature=1.0)